In [ ]:
import xmlrpc.client
import pymssql
import schedule
import time

# Odoo Connection Details
odoo_url = 'http://144.76.159.183:8069'
db_name = 'Backup_20250310'
username = 'admin'
password = 'admin'

# SQL Server Connection Details
server_name = 'SARAH\\SQLEXPRESS'  
database_name = 'Odoo_sql_database'   
sql_user_name = 'SuperAdmin'   
sql_password = 'SuperAdmin'    

def sync_customers():
    print("Starting synchronization of customer data...")

    # Connect to Odoo
    common = xmlrpc.client.ServerProxy(f'{odoo_url}/xmlrpc/2/common')
    uid = common.authenticate(db_name, username, password, {})

    if not uid:
        print("Failed to authenticate. Check your credentials.")
        return

    print(f"Authenticated successfully. User ID: {uid}")

    # Define the 'models' object
    models = xmlrpc.client.ServerProxy(f'{odoo_url}/xmlrpc/2/object')

    # Fetch only the required fields (id and name)
    fields = ['id', 'name']  

    try:
        customer_data = models.execute_kw(
            db_name,
            uid,
            password,
            'res.partner',  # Odoo model for customers
            'search_read',
            [[]],  # Fetch all customers
            {'fields': fields}
        )

        if not customer_data:
            print("No customer data fetched. Exiting synchronization.")
            return

        print(f"Fetched {len(customer_data)} customer records from Odoo.")

        # Insert customer data into MS SQL Server
        try:
            # Connect to SQL Server
            conn = pymssql.connect(server=server_name, user=sql_user_name, password=sql_password, database=database_name)
            cursor = conn.cursor()

            # Insert data into the CustomerData table
            for record in customer_data:
                customer_id = record.get('id')
                customer_name = record.get('name', '')

                # Insert only customer_id and customer_name
                cursor.execute("""
                    INSERT INTO CustomerData (customer_id, customer_name)
                    VALUES (%s, %s)
                """, (customer_id, customer_name))

            # Commit changes
            conn.commit()
            print(f"Inserted {len(customer_data)} customers into the SQL Server database.")

            # Close the connection
            cursor.close()
            conn.close()

        except pymssql.Error as e:
            print(f"Error while connecting to SQL Server or executing query: {e}")

    except xmlrpc.client.Fault as e:
        print(f"Error fetching customer data from Odoo: {e}")

# Schedule the function to run every 4 hours
schedule.every(4).hours.do(sync_customers)

# Run the function immediately
sync_customers()

print("Scheduler started. The script will also run every 4 hours.")

# Keep the script running for scheduled tasks
while True:
    schedule.run_pending()
    time.sleep(1)


Starting synchronization of customer data...
Authenticated successfully. User ID: 112
Fetched 841 customer records from Odoo.
Inserted 841 customers into the SQL Server database.
Scheduler started. The script will also run every 4 hours.
